In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast

In [3]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [4]:
!pip install wandb

In [5]:
import wandb
wandb.login()

wandb: Currently logged in as: rochelleli (use `wandb login --relogin` to force relogin)


True

In [6]:
wandb.init(project="w251-hw9", entity="rochelleli")

In [7]:
SEED=2

random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [8]:
torch.cuda.device_count()

1

In [9]:
START_EPOCH = 0

In [10]:
ARCH = 'resnet18'
EPOCHS = 2
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
PRINT_FREQ = 10
TRAIN_BATCH=500
VAL_BATCH=500
WORKERS=2
TRAINDIR="/home/ubuntu/data/train"
VALDIR="/home/ubuntu/data/val"

In [11]:
if not torch.cuda.is_available():
    print('GPU not detected.. did you pass through your GPU?')

In [12]:
wandb.init(config={"epochs": EPOCHS, "batch_size": TRAIN_BATCH, "momentum": MOMENTUM, "WEIGHT_DECAY": WEIGHT_DECAY, "arch": ARCH})

In [13]:
GPU = 0

In [14]:
torch.cuda.set_device(GPU)

In [15]:
global_step = 0

In [16]:
def train(train_loader, model, criterion, optimizer, epoch):
    global global_step    
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # Grad Scaler
    scaler = GradScaler()
    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        optimizer.zero_grad()

        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)

        # compute output
        with autocast():
          output = model(images)
          loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        
        # use the scaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
        writer.add_scalar("Loss/train", loss, global_step = global_step)
        writer.add_scalar("acc1/train", top1.avg, global_step = global_step)
        writer.add_scalar("acc5/train", top5.avg, global_step = global_step)
        
        wandb.log({"Loss/train": loss, 'acc1/train': top1.avg, 'acc5/train': top5.avg})
        
        global_step = global_step + 1

        if i % PRINT_FREQ == 0:
            progress.display(i)

In [17]:
def validate(val_loader, model, criterion):
    global global_step    
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
    writer.add_scalar("Loss/val", losses.avg, global_step = global_step)
    writer.add_scalar("acc1/val", top1.avg, global_step = global_step)
    writer.add_scalar("acc5/val", top5.avg, global_step = global_step)    
    
    wandb.log({"Loss/val": losses.avg, 'acc1/val': top1.avg, 'acc5/val': top5.avg})
    
    global_step = global_step + 1

    return top1.avg

In [18]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [19]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [20]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [21]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [22]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [23]:
cudnn.benchmark = True

In [24]:
import torch.distributed as dist
import sys

In [25]:
WORLD_SIZE = 2
BACKEND = 'nccl'

URL = 'tcp://172.31.17.123:443'

RANK = 1

dist.init_process_group(backend = BACKEND, init_method=URL,rank=RANK, world_size=WORLD_SIZE)

In [26]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]

In [27]:
normalize = transforms.Normalize(mean=imagenet_mean_RGB, std=imagenet_std_RGB)

In [28]:
IMG_SIZE = 224
NUM_CLASSES = 1000

In [29]:
model = models.__dict__[ARCH]()

In [30]:
inf = model.fc.in_features

In [31]:
model.fc = nn.Linear(inf, NUM_CLASSES)

In [32]:
model.cuda(GPU)
model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[GPU])

In [33]:
criterion = nn.CrossEntropyLoss().cuda(GPU)

In [34]:
optimizer = torch.optim.SGD(model.parameters(), LR,
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

In [35]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [36]:
transform_train = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomCrop(IMG_SIZE, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [37]:
train_dataset = datasets.ImageFolder(
    TRAINDIR, transform=transform_train)

In [38]:
transform_val = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.RandomCrop(IMG_SIZE, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_mean_RGB, imagenet_std_RGB),
])

In [39]:
val_dataset = datasets.ImageFolder(
    VALDIR, transform=transform_val)

In [40]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=TRAIN_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=torch.utils.data.distributed.DistributedSampler(train_dataset))

In [41]:
val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=VAL_BATCH, shuffle=False,
        num_workers=WORKERS, pin_memory=True, sampler=None)

In [42]:
best_acc1 = 0

In [43]:
for epoch in range(START_EPOCH, 2):
#    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()[0]))
    
    writer.add_scalar("lr", scheduler.get_last_lr()[0], global_step = global_step)
    
    wandb.log({'lr': scheduler.get_last_lr()[0]})

Epoch: [0][   0/1282]	Time 15.616 (15.616)	Data  4.128 ( 4.128)	Loss 7.0156e+00 (7.0156e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.60 (  0.60)
Epoch: [0][  10/1282]	Time  0.785 ( 2.468)	Data  0.127 ( 0.819)	Loss 6.9182e+00 (6.9759e+00)	Acc@1   0.00 (  0.11)	Acc@5   0.80 (  0.64)
Epoch: [0][  20/1282]	Time  0.754 ( 2.112)	Data  0.002 ( 0.916)	Loss 6.9132e+00 (6.9532e+00)	Acc@1   0.60 (  0.17)	Acc@5   1.80 (  0.84)
Epoch: [0][  30/1282]	Time  0.764 ( 1.991)	Data  0.003 ( 0.945)	Loss 6.8345e+00 (6.9262e+00)	Acc@1   0.00 (  0.23)	Acc@5   2.00 (  1.00)
Epoch: [0][  40/1282]	Time  0.758 ( 1.902)	Data  0.002 ( 0.938)	Loss 6.7156e+00 (6.8819e+00)	Acc@1   0.60 (  0.38)	Acc@5   3.20 (  1.40)
Epoch: [0][  50/1282]	Time  0.667 ( 1.863)	Data  0.003 ( 0.948)	Loss 6.6571e+00 (6.8398e+00)	Acc@1   0.60 (  0.46)	Acc@5   2.00 (  1.69)
Epoch: [0][  60/1282]	Time  0.695 ( 1.849)	Data  0.002 ( 0.968)	Loss 6.5180e+00 (6.7952e+00)	Acc@1   0.60 (  0.52)	Acc@5   3.40 (  1.95)
Epoch: [0][  70/1282]	Time  0.755 ( 1.820

Epoch: [0][ 600/1282]	Time  0.680 ( 1.704)	Data  0.002 ( 0.988)	Loss 4.7113e+00 (5.4627e+00)	Acc@1  12.40 (  6.34)	Acc@5  28.60 ( 16.92)
Epoch: [0][ 610/1282]	Time  0.679 ( 1.703)	Data  0.002 ( 0.988)	Loss 4.6303e+00 (5.4466e+00)	Acc@1  15.00 (  6.47)	Acc@5  29.60 ( 17.18)
Epoch: [0][ 620/1282]	Time  0.781 ( 1.703)	Data  0.002 ( 0.987)	Loss 4.4353e+00 (5.4315e+00)	Acc@1  11.40 (  6.57)	Acc@5  35.20 ( 17.41)
Epoch: [0][ 630/1282]	Time  0.680 ( 1.703)	Data  0.002 ( 0.987)	Loss 4.3729e+00 (5.4163e+00)	Acc@1  16.00 (  6.70)	Acc@5  36.20 ( 17.67)
Epoch: [0][ 640/1282]	Time  0.687 ( 1.702)	Data  0.002 ( 0.986)	Loss 4.5844e+00 (5.4018e+00)	Acc@1  13.20 (  6.80)	Acc@5  32.00 ( 17.89)
Epoch: [0][ 650/1282]	Time  0.682 ( 1.702)	Data  0.002 ( 0.987)	Loss 4.3705e+00 (5.3878e+00)	Acc@1  16.60 (  6.91)	Acc@5  35.20 ( 18.14)
Epoch: [0][ 660/1282]	Time  0.685 ( 1.701)	Data  0.002 ( 0.986)	Loss 4.4043e+00 (5.3734e+00)	Acc@1  14.20 (  7.03)	Acc@5  33.60 ( 18.37)
Epoch: [0][ 670/1282]	Time  0.781 ( 1.701

Epoch: [0][1200/1282]	Time  0.778 ( 1.690)	Data  0.002 ( 0.968)	Loss 3.5538e+00 (4.7478e+00)	Acc@1  25.60 ( 12.86)	Acc@5  50.20 ( 29.13)
Epoch: [0][1210/1282]	Time  0.689 ( 1.691)	Data  0.002 ( 0.968)	Loss 3.7215e+00 (4.7378e+00)	Acc@1  21.00 ( 12.97)	Acc@5  48.00 ( 29.31)
Epoch: [0][1220/1282]	Time  0.688 ( 1.690)	Data  0.002 ( 0.968)	Loss 3.6594e+00 (4.7287e+00)	Acc@1  24.40 ( 13.06)	Acc@5  47.40 ( 29.48)
Epoch: [0][1230/1282]	Time  0.715 ( 1.690)	Data  0.002 ( 0.968)	Loss 3.6392e+00 (4.7193e+00)	Acc@1  24.40 ( 13.16)	Acc@5  47.60 ( 29.64)
Epoch: [0][1240/1282]	Time  0.713 ( 1.691)	Data  0.002 ( 0.969)	Loss 3.5611e+00 (4.7102e+00)	Acc@1  25.60 ( 13.25)	Acc@5  51.40 ( 29.80)
Epoch: [0][1250/1282]	Time  0.723 ( 1.690)	Data  0.002 ( 0.969)	Loss 3.4320e+00 (4.7010e+00)	Acc@1  28.20 ( 13.35)	Acc@5  51.80 ( 29.96)
Epoch: [0][1260/1282]	Time  0.721 ( 1.691)	Data  0.003 ( 0.969)	Loss 3.6723e+00 (4.6922e+00)	Acc@1  24.20 ( 13.45)	Acc@5  47.20 ( 30.11)
Epoch: [0][1270/1282]	Time  0.784 ( 1.690

Epoch: [1][ 430/1282]	Time  2.579 ( 1.673)	Data  0.002 ( 0.040)	Loss 3.0770e+00 (3.2083e+00)	Acc@1  34.40 ( 31.57)	Acc@5  61.40 ( 57.16)
Epoch: [1][ 440/1282]	Time  2.591 ( 1.673)	Data  0.002 ( 0.039)	Loss 2.9669e+00 (3.2056e+00)	Acc@1  34.40 ( 31.61)	Acc@5  61.60 ( 57.22)
Epoch: [1][ 450/1282]	Time  2.437 ( 1.674)	Data  0.002 ( 0.038)	Loss 2.9060e+00 (3.2036e+00)	Acc@1  34.20 ( 31.64)	Acc@5  60.40 ( 57.24)
Epoch: [1][ 460/1282]	Time  2.657 ( 1.674)	Data  0.002 ( 0.038)	Loss 3.1396e+00 (3.2007e+00)	Acc@1  30.80 ( 31.69)	Acc@5  59.00 ( 57.29)
Epoch: [1][ 470/1282]	Time  2.674 ( 1.675)	Data  0.002 ( 0.037)	Loss 3.1846e+00 (3.1978e+00)	Acc@1  30.80 ( 31.72)	Acc@5  57.40 ( 57.34)
Epoch: [1][ 480/1282]	Time  2.767 ( 1.675)	Data  0.002 ( 0.036)	Loss 2.9806e+00 (3.1947e+00)	Acc@1  36.00 ( 31.78)	Acc@5  59.80 ( 57.40)
Epoch: [1][ 490/1282]	Time  2.575 ( 1.675)	Data  0.003 ( 0.035)	Loss 3.0998e+00 (3.1925e+00)	Acc@1  33.80 ( 31.81)	Acc@5  58.80 ( 57.44)
Epoch: [1][ 500/1282]	Time  2.951 ( 1.676

Epoch: [1][1030/1282]	Time  2.512 ( 1.675)	Data  0.002 ( 0.018)	Loss 2.7990e+00 (3.0591e+00)	Acc@1  39.00 ( 34.01)	Acc@5  65.00 ( 59.84)
Epoch: [1][1040/1282]	Time  2.765 ( 1.675)	Data  0.002 ( 0.018)	Loss 2.7864e+00 (3.0570e+00)	Acc@1  36.00 ( 34.05)	Acc@5  67.20 ( 59.88)
Epoch: [1][1050/1282]	Time  2.521 ( 1.674)	Data  0.002 ( 0.018)	Loss 2.7155e+00 (3.0544e+00)	Acc@1  40.40 ( 34.10)	Acc@5  67.80 ( 59.93)
Epoch: [1][1060/1282]	Time  2.503 ( 1.674)	Data  0.003 ( 0.018)	Loss 2.7283e+00 (3.0518e+00)	Acc@1  39.00 ( 34.13)	Acc@5  67.60 ( 59.99)
Epoch: [1][1070/1282]	Time  2.816 ( 1.674)	Data  0.002 ( 0.018)	Loss 2.8005e+00 (3.0500e+00)	Acc@1  37.80 ( 34.16)	Acc@5  62.60 ( 60.01)
Epoch: [1][1080/1282]	Time  2.694 ( 1.674)	Data  0.002 ( 0.017)	Loss 2.7506e+00 (3.0478e+00)	Acc@1  42.00 ( 34.20)	Acc@5  64.40 ( 60.05)
Epoch: [1][1090/1282]	Time  2.512 ( 1.673)	Data  0.002 ( 0.017)	Loss 2.7566e+00 (3.0456e+00)	Acc@1  38.40 ( 34.24)	Acc@5  65.00 ( 60.09)
Epoch: [1][1100/1282]	Time  2.546 ( 1.673

In [44]:
writer.close()
%load_ext tensorboard
%tensorboard --logdir=runs